# Working with Geolocated Data
---

### Exercise: Earthquake Monitor

Collect the data:

In [32]:
import requests

endpoint = "https://earthquake.usgs.gov/fdsnws/event/1/query?"

ps = { "format": "csv", 
       "starttime": "2025-09-01", 
       "endtime": "2025-09-26",
       "minmagnitude": 5.5
     }

response = requests.get(endpoint, params=ps)

import pandas as pd
import io

buf = io.StringIO(response.text) 
data = pd.read_csv(buf)

The available information about each earthquake:

In [33]:
for col in data.columns:
    print(col)

time
latitude
longitude
depth
mag
magType
nst
gap
dmin
rms
net
id
updated
place
type
horizontalError
depthError
magError
magNst
status
locationSource
magSource


Draw the map:

In [34]:
import folium

greenwich_coords = [51.4769, 0]  # i.e. [Latitude, Longitude]

world_map = folium.Map(location=greenwich_coords,
                                zoom_start = 1)

# Add markers
for i in range(len(data)):
    quake = data.iloc[i,:]
    folium.Marker(location=[quake.latitude,quake.longitude], 
                  popup = quake.mag
                 ).add_to(world_map)


world_map

Again but with `folium.CircleMarker`:

In [35]:
import folium

greenwich_coords = [51.4769, 0]  # i.e. [Latitude, Longitude]

world_map = folium.Map(location=greenwich_coords,
                                zoom_start = 1)

# Add markers
for i in range(len(data)):
    quake = data.iloc[i,:]
    folium.CircleMarker(location=[quake.latitude,quake.longitude], 
                        popup = quake.mag,
                        weight=1,
                        fill=True,
                        fill_opacity=0.3,
                        radius = quake.mag * 2
                       ).add_to(world_map)


world_map

For the colouring, we need to request the GeoJSON data:

In [36]:
endpoint = "https://earthquake.usgs.gov/fdsnws/event/1/query?"

ps = { "format": "geojson", 
       "starttime": "2025-09-01", 
       "endtime": "2025-09-26",
       "minmagnitude": 5.5 
     }

response = requests.get(endpoint, params=ps)

import json
data_dict = json.loads(response.text)


Let's inspect the first earthquake record:

In [37]:
quake = data_dict['features'][0]
quake

{'type': 'Feature',
 'properties': {'mag': 5.9,
  'place': '90 km E of Ozernovskiy, Russia',
  'time': 1758293729171,
  'updated': 1758295581015,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/us7000qxcd',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=us7000qxcd&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': 4.499,
  'alert': 'green',
  'status': 'reviewed',
  'tsunami': 0,
  'sig': 536,
  'net': 'us',
  'code': '7000qxcd',
  'ids': ',usauto7000qxcd,pt25262001,us7000qxcd,',
  'sources': ',usauto,pt,us,',
  'types': ',internal-moment-tensor,internal-origin,losspager,moment-tensor,origin,phase-data,shakemap,',
  'nst': 62,
  'dmin': 1.688,
  'rms': 0.89,
  'gap': 116,
  'magType': 'mww',
  'type': 'earthquake',
  'title': 'M 5.9 - 90 km E of Ozernovskiy, Russia'},
 'geometry': {'type': 'Point', 'coordinates': [157.7944, 51.422, 84.495]},
 'id': 'us7000qxcd'}

Notice that the position is given by

In [38]:
quake['geometry']['coordinates']

[157.7944, 51.422, 84.495]

which is in the format [longitude, latitude, depth].

The other information can be retrieved as e.g.

In [39]:
quake['properties']['mag']

5.9

In [40]:
import folium

greenwich_coords = [51.4769, 0]  # i.e. [Latitude, Longitude]

world_map = folium.Map(location=greenwich_coords,
                                zoom_start = 1)

# Add markers
for quake in data_dict['features']:
    coords = quake['geometry']['coordinates']
    folium.CircleMarker(location=[coords[1],coords[0]], 
                        popup = quake['properties']['mag'],
                        weight=1,
                        fill=True,
                        fill_opacity=0.3,
                        color=quake['properties']['alert'],
                        radius = quake['properties']['mag'] * 2
                       ).add_to(world_map)


world_map